In [ ]:
#@title Convert all Metro PDF's to excel
# create folder metro/pdfs from your application root and upload pdfs to it

try:
  import os
  import fitz
  import tabula
  import pandas as pd
  from pathlib import Path
  import sys
except Exception as e:
  !pip install PyMuPDF==1.23.7
  !pip install -q tabula-py==2.9.0
finally:
  import os
  import fitz
  import tabula
  import pandas as pd
  from pathlib import Path
  import sys


Path("/content/metro/pdfs/").mkdir(parents=True, exist_ok=True)
output_filename = "metro_combined_orders.csv"
pdf_source_path = "/content/metro/pdfs/"
for d in os.listdir(pdf_source_path):
  if os.path.isdir(os.path.join(pdf_source_path,d)):
    os.rmdir(os.path.join(pdf_source_path,d))
input("upload pdfs hit enter")
# Output Columns List
columns = [
    "Supplier no",
    "Order no",
    "Delivery date from",
    "Delivery date to",
    "Order for Store Number",
    "Article Number",
    "Subsystem number",
    "Your art-no.",
    "Sales-EAN",
    "Article Name",
    "HSN",
    "Variant",
    "MRP INR",
    "Buy Bdl",
    "Sales BDL",
    "SGST %",
    "CGST %",
    "IGST %",
    "CESS %",
    "Qty Buy Bdl",
    "Qty sale Bdl",
    "Cess INR",
    "Total NBP INR",
    "NBP pP amt",
    "Total Order Amount",
    "Total order amount INR",
]
#common indexmap
indexmap = {
    "Article Number": [1, 2],
    "Subsystem number": 3,
    "Your art-no.": 4,
    "Sales-EAN": 14,
    "Article Name": [5, 15],
    "HSN": 20,
    "Variant": 6,
    "MRP INR": 21,
    "Buy Bdl": [10, 12],
    "Sales BDL": [11, 13],
    "SGST %": 0,
    "CGST %": 0,
    "IGST %": 23,
    "CESS %": 0,
    "Qty Buy Bdl": 8,
    "Qty sale Bdl": 7,
    "Cess INR": 0,
    "Total NBP INR": 9,
    "NBP pP amt": 17,
}

def divide_chunks(l, n):
    # Dynamic list split
    for i in range(0, len(l), n):
        yield l.iloc[i : i + n]

def page_formettor(pageno, df):
  # formet table according to output format
    import re

    df_trans = []

    trans = list(divide_chunks(df, 3))
    if pageno == 0:
        for x in trans:
            indexmap = {}
            indexmap["Article Number"] = x.iloc[0].iloc[1]
            indexmap["Subsystem number"] = x.iloc[1].iloc[1]
            if (
                re.search(
                    r"^[A-Z]{2}\-[A-Z]{2}\s[A-Z]{1}\s\d{1}[A-Z]{1}\d{2}",
                    x.iloc[0].iloc[3],
                )
                or re.search(
                    r"^[A-Z]{2}\-[A-Z]{2}\s[A-Z]{1}\s\d{2}[A-Z]{1}\d{1}",
                    x.iloc[0].iloc[3],
                )
                or re.search(
                    r"^[A-Z]{2}\-[A-Z]{2}\s[A-Z]{1}\s\d{1}[A-Z]{1}", x.iloc[0].iloc[3]
                )
            ):
                indexmap["Your art-no."] = x.iloc[0].iloc[1].split(" ")[0]
            else:
                indexmap["Your art-no."] = x.iloc[0].iloc[3]
            indexmap["Sales-EAN"] = x.iloc[1].iloc[3]
            if type(x.iloc[1].iloc[4]) != float:
                indexmap["Article Name"] = x.iloc[0].iloc[4] + x.iloc[1].iloc[4]
            else:
                indexmap["Article Name"] = x.iloc[0].iloc[4]
            indexmap["HSN"] = x.iloc[2].iloc[4]
            indexmap["Variant"] = x.iloc[0].iloc[6].split(" ")[0]
            if re.search(r"^\d{2}\.\d{2}%", x.iloc[2].iloc[6].split(" ")[0]):
                indexmap["MRP INR"] = ""
            else:
                indexmap["MRP INR"] = x.iloc[2].iloc[6].split(" ")[0]
            indexmap["Buy Bdl"] = (
                x.iloc[0].iloc[6].split(" ")[1] + x.iloc[0].iloc[6].split(" ")[2]
            )
            indexmap["Sales BDL"] = (
                x.iloc[1].iloc[6].split(" ")[0] + x.iloc[1].iloc[6].split(" ")[1]
            )
            indexmap["SGST %"] = ""
            indexmap["CGST %"] = ""
            indexmap["IGST %"] = x.iloc[2].iloc[6].split(" ")[2]
            indexmap["CESS %"] = ""
            indexmap["Qty Buy Bdl"] = x.iloc[0].iloc[6].split(" ")[4]
            indexmap["Qty sale Bdl"] = x.iloc[1].iloc[6].split(" ")[3]
            indexmap["Cess INR"] = ""
            indexmap["Total NBP INR"] = x.iloc[0].iloc[6].split(" ")[-1]
            indexmap["NBP pP amt"] = x.iloc[1].iloc[6].split(" ")[-1]
            df_trans.append(indexmap)
    else:
        for x in trans:
            if len(x) == 3:
                indexmap = {}
                indexmap["Article Number"] = x.iloc[0].iloc[1]
                indexmap["Subsystem number"] = x.iloc[1].iloc[1]
                if (
                    re.search(
                        r"^[A-Z]{2}\-[A-Z]{2}\s[A-Z]{1}\s\d{1}[A-Z]{1}\d{2}",
                        x.iloc[0].iloc[3],
                    )
                    or re.search(
                        r"^[A-Z]{2}\-[A-Z]{2}\s[A-Z]{1}\s\d{2}[A-Z]{1}\d{1}",
                        x.iloc[0].iloc[3],
                    )
                    or re.search(
                        r"^[A-Z]{2}\-[A-Z]{2}\s[A-Z]{1}\s\d{1}[A-Z]{1}",
                        x.iloc[0].iloc[3],
                    )
                ):
                    indexmap["Your art-no."] = x.iloc[0].iloc[1].split(" ")[0]
                else:
                    indexmap["Your art-no."] = x.iloc[0].iloc[3]
                indexmap["Sales-EAN"] = x.iloc[1].iloc[3]
                if type(x.iloc[1].iloc[4]) != float:
                    indexmap["Article Name"] = x.iloc[0].iloc[4] + x.iloc[1].iloc[4]
                else:
                    indexmap["Article Name"] = x.iloc[0].iloc[4]
                indexmap["HSN"] = x.iloc[2].iloc[4]
                indexmap["Variant"] = x.iloc[0].iloc[5]
                if re.search(r"^\d{2}\.\d{2}%", str(x.iloc[2].iloc[5])):
                    indexmap["MRP INR"] = ""
                else:
                    indexmap["MRP INR"] = x.iloc[2].iloc[5]
                indexmap["Buy Bdl"] = x.iloc[0].iloc[6]
                indexmap["Sales BDL"] = x.iloc[1].iloc[6]
                indexmap["SGST %"] = ""
                indexmap["CGST %"] = ""
                if type(x.iloc[2].iloc[8]) != float:
                    indexmap["IGST %"] = x.iloc[2].iloc[8]
                else:
                    indexmap["IGST %"] = x.iloc[2].iloc[7]
                indexmap["CESS %"] = ""
                indexmap["Qty Buy Bdl"] = x.iloc[0].iloc[10].split(" ")[0]
                indexmap["Qty sale Bdl"] = x.iloc[1].iloc[10].split(" ")[0]
                indexmap["Cess INR"] = ""
                indexmap["Total NBP INR"] = x.iloc[0].iloc[10].split(" ")[-1]
                indexmap["NBP pP amt"] = x.iloc[1].iloc[10].split(" ")[-1]
                df_trans.append(indexmap)
    return pd.DataFrame.from_records(df_trans)

def get_trans_df(dfs):
  # get Trans dataframes
    trans_df = []
    for i, x in enumerate(dfs):
        if i == 0:
            get_all_indexs = page_formettor(i, x.iloc[5:])
        else:
            get_all_indexs = page_formettor(i, x.iloc[2:])
        trans_df.append(get_all_indexs)
    return trans_df

# getting Common Row Data
def get_row_data(pdffolderpath, pdfreader, path=False):
    missing = None
    pdfpath = []
    if missing is None:
        pdfs = os.listdir(pdffolderpath)
        files = []
        for i in pdfs:
            fit = pdfreader.open("".join([pdffolderpath, i]))
            if path:
                pdfpath.append("".join([pdffolderpath, i]))
            pages = []
            for page in fit:
                pages.append(page.get_text())
            files.append({"".join([pdffolderpath, i]): pages[:-1]})
    else:
        files = []
        pdfs = []
        for id in missing:
            for sed in os.listdir(pdffolderpath):
                if id in sed:
                    pdfs.append(sed)
        for i in pdfs:
            fit = pdfreader.open("".join([pdffolderpath, i]))
            pages = []
            for page in fit:
                pages.append(page.get_text())
            files.append({"".join([pdffolderpath, i]): pages[:-1]})
    if path:
        return files, pdfpath
    else:
        return files

def get_pdf_folder_done(folder_path):
  # get all Pdfs and tables
    total_df_data = []
    for i in os.listdir(folder_path):
        dfs = tabula.read_pdf(os.path.join(folder_path, i), pages="all")
        df = pd.concat(get_trans_df(dfs))
        df["file"] = os.path.join(folder_path, i)
        total_df_data.append(df)
    return pd.concat(total_df_data)

files = get_row_data(pdf_source_path, fitz)
df    = get_pdf_folder_done(pdf_source_path)

invoices = []
for i, v in enumerate(files):
    file, v = list(v.keys())[0], list(v.values())[0]
    fipag = []
    invdetails = {}
    for q, c in enumerate(v):
        if q == 0:
            if [x for x in v[q].split("\n") if x.startswith("Supplier no")] != []:
                testa = [x for x in v[q].split("\n")]
                indexs = testa.index(
                    [x for x in v[q].split("\n") if x.startswith("Supplier no")][0]
                )
                invdetails["Supplier no"] = testa[indexs + 1]
            if [x for x in v[q].split("\n") if x.startswith("Order no")] != []:
                testa = [x for x in v[q].split("\n")]
                indexs = testa.index(
                    [x for x in v[q].split("\n") if x.startswith("Order no")][0]
                )
                invdetails["Order no"] = testa[indexs + 1]
            if [
                x for x in v[q].split("\n") if x.startswith("Delivery date from")
            ] != []:
                testa = [x for x in v[q].split("\n")]
                indexs = testa.index(
                    [x for x in v[q].split("\n") if x.startswith("Delivery date from")][
                        0
                    ]
                )
                invdetails["Delivery date from"] = testa[indexs - 1]
            if [x for x in v[q].split("\n") if x.startswith("to")] != []:
                testa = [x for x in v[q].split("\n")]
                indexs = testa.index(
                    [x for x in v[q].split("\n") if x.startswith("to")][0]
                )
                invdetails["Delivery date to"] = testa[indexs - 1]
            if [
                x for x in v[q].split("\n") if x.startswith("Order for Store no")
            ] != []:
                testa = [x for x in v[q].split("\n")]
                indexs = testa.index(
                    [x for x in v[q].split("\n") if x.startswith("Order for Store no")][
                        0
                    ]
                )
                invdetails["Order for Store Number"] = (
                    testa[indexs].split(":")[-1].strip()
                )
            valid = [q for x in c.split("\n") if "Total order amount" in x]
            if valid == []:
                fipag.append(c.split("\n")[57:])
            else:
                raise NotImplementedError("data sample not available")
        else:
            data = None
            valid = [q for x in c.split("\n") if "Total order amount" in x]
            if valid == []:
                fipag.append(c.split("\n")[52:])
            else:
                try:
                    data = float(c.split("\n")[67:][0])
                except ValueError as e:
                    fipag.append(c.split("\n")[68:])
                if data is not None:
                    fipag.append(c.split("\n")[67:])
                invdetails["Total order amount"] = c.split("\n")[52:][0]
    invdetails["file"] = file
    invoices.append(invdetails)

invoicedf = pd.DataFrame.from_records(invoices)
records = []
# prepearing output
for k, v in df.iterrows():
    common_info = invoicedf[invoicedf["file"] == v["file"]]
    for x, d in common_info.iterrows():
        for xs in common_info.columns.to_list():
            v[xs] = d[xs]
    records.append(v.to_dict())

df = pd.DataFrame.from_records(records)
df=df.fillna(0)
try:
  dfd=df[df['Total order amount']==0]
except KeyError as E:
  df['Total order amount']=[0 for x in range(len(df))]
  dfd=df[df['Total order amount']==0]
filllastdata=[]
for k,v in dfd.iterrows():
  pdf=fitz.open(v['file'])
  content=[]
  for page in pdf:
    content.append(page.get_text())
  totalorder=content[-1].split("\n")
  if 'Total order amount INR' in totalorder:
    filllastdata.append((v['file'],totalorder[totalorder.index('Total order amount INR')-1]))
  else:
    totalorder=content[-2].split("\n")
    filllastdata.append((v['file'],totalorder[totalorder.index('Total order amount INR')-1]))

for x in filllastdata:
  df.loc[df['file']==x[0], 'Total order amount'] = x[1]

df = df.drop(columns=['file'])
df.to_csv(output_filename, index=False)

